In [1]:
# STEP 1: Install all the necessary libraries.
# LangChain for orchestration, LangChain-Groq for Groq LLM.
!pip install -q langchain langchain-groq langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [2]:
from langchain_core.output_parsers.json import JsonOutputParser

In [3]:
# STEP 2: Gather all the wands and scrolls we shall need.
from langchain_groq import ChatGroq
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory # <--- CORRECTED IMPORT
from IPython.display import Markdown, display
from google.colab import userdata
import os

In [4]:
# STEP 3: Whisper your secret key to the winds.
try:
    api_key = userdata.get("GROQ_API_KEY")
except Exception:
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise ValueError("GROQ_API_KEY not found. Please set it!")

# STEP 3b: Summon the Groq-powered LLM.
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=api_key,
    temperature=0.3,
)

In [5]:
# STEP 4: Forge a prompt that commands your LLM to obey the JSON contract.
# -------------------------------------------
# 💡 IMPORTANT:
# For JSONOutputParser to work, your prompt must tell the LLM to respond in JSON.
# Otherwise the parser will fail — because it expects valid JSON.
# -------------------------------------------

system_msg = SystemMessagePromptTemplate.from_template(
    "You are a structured data assistant. Always reply in valid JSON with keys: 'answer' and 'source'."
)

human_msg = HumanMessagePromptTemplate.from_template(
    "{input}"
)

chat_prompt = ChatPromptTemplate.from_messages([
    system_msg,
    MessagesPlaceholder(variable_name="history"),
    human_msg
])

In [6]:
# STEP 5: As before, keep your memories safe.
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


In [8]:
# STEP 6: 🌟 The heart of the magic.
# -------------------------------------------
# 💡 WHAT JsonOutputParser DOES:
# It expects the raw LLM output to be a valid JSON string.
# Then, it parses it into a Python dict.
# If the LLM outputs invalid JSON, the parser will raise an error — so your prompt must guide it carefully.
# -------------------------------------------

chat_chain = chat_prompt | llm

# Add the JsonOutputParser — it will parse text into a Python dict.
parsed_chain = chat_chain | JsonOutputParser()

# Wrap with memory support, so the chain remembers your tale.
chatbot = RunnableWithMessageHistory(
    parsed_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [9]:
# STEP 7: Test your chain.
# You should see neat Python dictionaries coming back.

session_id = "chat-session-json-001"

user_inputs = [
    "Explain what is LangChain, cite the official docs URL as source.",
    "What is JSONOutputParser used for? Cite the docs URL."
]

print(f"Starting chat loop with session ID: {session_id}")

for input_text in user_inputs:
    print(f"\nUser: {input_text}")
    response = chatbot.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}}
    )
    # The response will be a Python dict: {'answer': ..., 'source': ...}
    print("Parsed Response:", response)
    display(Markdown(f"**Answer:** {response['answer']}\n\n**Source:** {response['source']}"))

print("\n--- Stored Chat History ---")
for message in store[session_id].messages:
    print(f"{message.type.capitalize()}: {message.content}")


Starting chat loop with session ID: chat-session-json-001

User: Explain what is LangChain, cite the official docs URL as source.


Parsed Response: {'answer': 'LangChain is an open-source framework designed to help developers build applications powered by large language models. It provides a set of tools and libraries to simplify the process of integrating language models into various applications, allowing developers to focus on building their products rather than spending time on the underlying infrastructure.', 'source': 'https://langchain.readthedocs.io/en/latest/'}


**Answer:** LangChain is an open-source framework designed to help developers build applications powered by large language models. It provides a set of tools and libraries to simplify the process of integrating language models into various applications, allowing developers to focus on building their products rather than spending time on the underlying infrastructure.

**Source:** https://langchain.readthedocs.io/en/latest/


User: What is JSONOutputParser used for? Cite the docs URL.


Parsed Response: {'answer': 'JSONOutputParser is used for parsing JSON output from various sources, but the specific use case may vary depending on the context. Generally, it is utilized to extract and process data from JSON-formatted strings or files.', 'source': 'https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/regex/Pattern.html'}


**Answer:** JSONOutputParser is used for parsing JSON output from various sources, but the specific use case may vary depending on the context. Generally, it is utilized to extract and process data from JSON-formatted strings or files.

**Source:** https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/regex/Pattern.html


--- Stored Chat History ---
